In [1]:
# DEV_Stocks_Data_Calculate_Based_on_Daily_001_Thread.ipynb
# Create by GF 2025-11-27 22:32

In [2]:
import json
import sys
# ..................................................
import pandas      # Pandas 2.3.3
import sqlalchemy  # SQLAlchemy 2.0.36
# ..................................................
import PY3_PostgreSQL9_By_Psycopg2
import PY3_Text_Progress_Bar
# ..................................................
from Python3_Finance import PY3_Finance_Entanglement_Theory
from Python3_Finance import PY3_Finance_Indicator_EMA
from Python3_Finance import PY3_Finance_Indicator_KDJ
from Python3_Finance import PY3_Finance_Indicator_MACD
from Python3_Finance import PY3_Finance_Indicator_SMA
from Python3_Finance import PY3_Stocks_Data_Conv_Daily_to_Weekly
from Python3_Finance import PY3_Stocks_DataFrame_Preprocessing

In [3]:
# 读取 Finance Indicator Calculate Configuration (JSON 文件)
# ..................................................
FILE_Object = open("./APP_Finance_Calculate_Configuration.json", mode = 'r', encoding = "utf-8")
TEXT_Readed = FILE_Object.read()
FILE_Object.close()
DICT_Config = json.loads(TEXT_Readed)
# ..................................................
db_host     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("host", "unknow_host")
db_port     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("port", "unknow_port")
db_name     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("init_database", "unknow_db_name")
db_user     = DICT_Config.get("postgresql", {}).get("16.10", {}).get("user", "unknow_user")
db_password = DICT_Config.get("postgresql", {}).get("16.10", {}).get("password", "unknow_password")
# ..................................................
stocks_mixed_db_table = DICT_Config.get("options", {}).get("stocks", {}).get("mixed", {}).get("db_table", "unkonw_db_table")
stocks_mixed_adjusted = DICT_Config.get("options", {}).get("stocks", {}).get("mixed", {}).get("adjusted", "unkonw_adjusted")

In [4]:
Text_Progress_Bar = PY3_Text_Progress_Bar.PY3_Text_Progress_Bar()
# ..................................................
Fin_Etg_Theo        = PY3_Finance_Entanglement_Theory.PY3_Finance_Entanglement_Theory()
Fin_Ind_EMA_for_012 = PY3_Finance_Indicator_EMA.PY3_Finance_Indicator_EMA()
Fin_Ind_EMA_for_026 = PY3_Finance_Indicator_EMA.PY3_Finance_Indicator_EMA()
Fin_Ind_KDJ         = PY3_Finance_Indicator_KDJ.PY3_Finance_Indicator_KDJ()
Fin_Ind_MACD        = PY3_Finance_Indicator_MACD.PY3_Finance_Indicator_MACD()
Fin_Ind_SMA_for_005 = PY3_Finance_Indicator_SMA.PY3_Finance_Indicator_SMA()
Fin_Ind_SMA_for_010 = PY3_Finance_Indicator_SMA.PY3_Finance_Indicator_SMA()
# ..................................................
Stocks_Data_Conv_Daily_to_Weekly = PY3_Stocks_Data_Conv_Daily_to_Weekly.PY3_Stocks_Data_Conv_Daily_to_Weekly()
Stocks_DataFrame_Preprocessing   = PY3_Stocks_DataFrame_Preprocessing.PY3_Stocks_DataFrame_Preprocessing()
# ..................................................
PGSQL = PY3_PostgreSQL9_By_Psycopg2.PY3_PostgreSQL9_By_Psycopg2()
PGSQL.Set_DB_Host(db_host)
PGSQL.Set_DB_Database(db_name)
PGSQL.Set_DB_User(db_user)
PGSQL.Set_DB_Password(db_password)

1

In [5]:
SQL_Filtering:str = "POSITION('股票' IN memo) > 0 AND POSITION('日数据' IN memo) > 0 AND POSITION('%s' IN memo) > 0" % stocks_mixed_adjusted
SQL_Statement:str = "SELECT * FROM %s WHERE %s;" % (stocks_mixed_db_table, SQL_Filtering)
df_stocks_daily_bfq = PGSQL.Query(SQL_Statement)
df_stocks_daily_bfq = df_stocks_daily_bfq.rename(columns = {"ts_code": "code", "pct_chg": "chg_pct", "vol": "volume"})
df_stocks_daily_bfq

,id,unique,code,time,open,high,low,close,pre_close,change,chg_pct,volume,amount,memo
0,978374,stock-300135-SZ-daily-20250930-bfq,300135.SZ,2025-09-30,4.4100,4.4300,4.3500,4.3600,4.3900,-0.0300,-0.6834,198088.0000,86594.1000,股票/日数据/不复权
1,978375,stock-300136-SZ-daily-20250930-bfq,300136.SZ,2025-09-30,25.8500,26.2400,25.4100,25.8000,25.7800,0.0200,0.0776,291497.0000,749882.0000,股票/日数据/不复权
2,978376,stock-300137-SZ-daily-20250930-bfq,300137.SZ,2025-09-30,6.6000,6.7900,6.5500,6.7500,6.6300,0.1200,1.8100,54999.6000,36986.9000,股票/日数据/不复权
3,978377,stock-300138-SZ-daily-20250930-bfq,300138.SZ,2025-09-30,13.0000,13.2300,12.9300,13.1100,12.9600,0.1500,1.1574,49482.6000,64794.7000,股票/日数据/不复权
4,978378,stock-300139-SZ-daily-20250930-bfq,300139.SZ,2025-09-30,28.0500,28.8000,27.7500,27.8100,27.6700,0.1400,0.5060,654474.0000,1846360.0000,股票/日数据/不复权
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319383,1217858,stock-300563-SZ-daily-20251209-bfq,300563.SZ,2025-12-09,40.9000,40.9000,38.6600,38.6600,39.7900,-1.1300,-2.8399,127903.3400,503324.0500,股票/日数据/不复权
1319384,1217859,stock-300564-SZ-daily-20251209-bfq,300564.SZ,2025-12-09,15.7700,15.7700,15.5100,15.6100,15.8000,-0.1900,-1.2025,12314.0000,19229.0560,股票/日数据/不复权
1319385,1217860,stock-300565-SZ-daily-20251209-bfq,300565.SZ,2025-12-09,13.0000,13.2000,12.8200,12.9200,13.0600,-0.1400,-1.0720,117297.5600,151955.4250,股票/日数据/不复权
1319386,1217861,stock-300566-SZ-daily-20251209-bfq,300566.SZ,2025-12-09,18.8500,18.9600,18.7000,18.7700,18.8200,-0.0500,-0.2657,42920.2900,80872.1160,股票/日数据/不复权


In [6]:
def Finance_Calculate_Stocks_Based_on_Daily(
    Pandas_DataFrame_for_Stocks_Daily,
    DB_TABLE_Stocks_Data_Daily:str            = "tushare_api_cache_en_us",
    DB_TABLE_Stocks_Data_Daily_Indicator:str  = "tushare_api_cache_en_us_indicator",
    DB_TABLE_Stocks_Data_Weekly:str           = "stocks_en_us_daily_to_weekly",
    DB_TABLE_Stocks_Data_Weekly_Indicator:str = "stocks_en_us_daily_to_weekly_indicator",
    DB_TABLE_Stocks_Data_Weekly_Interim:str   = "stocks_en_us_daily_to_weekly_interim",
    Stocks_Data_Adjusted = "bfq"):

    Adjusted:str = "不复权"
    if (Stocks_Data_Adjusted == "bfq"): Adjusted = "不复权"
    if (Stocks_Data_Adjusted == "hfq"): Adjusted = "后复权"
    if (Stocks_Data_Adjusted == "qfq"): Adjusted = "前复权"

    ########################################
    ## Calculating Stocks Daily Indicator ##
    ########################################

    df = Pandas_DataFrame_for_Stocks_Daily.copy()

    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator...\n""")
    df = Stocks_DataFrame_Preprocessing.Before_Calculating_Stocks_Daily_Indicators(df)
    # ..............................................
    JSON_Records_Stocks_Daily_Indicator:list = []
    # ..............................................
    Total = df["id"].count()
    Count = 1
    # ..............................................
    for Idx, Row in df.iterrows():
        ID            = Row["id"]
        Seq           = Row["row_num"]
        High          = Row["high"]
        Low           = Row["low"]
        Close         = Row["close"]
        # ..........................................
        if (Count == Total or Count % 100 == 0):
            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
            sys.stdout.flush()
        # ..........................................
        EMA12       = Fin_Ind_EMA_for_012.EMA(Seq = Seq, Period = 12, Close = Close)
        EMA26       = Fin_Ind_EMA_for_026.EMA(Seq = Seq, Period = 26, Close = Close)
        MACD_DIF    = Fin_Ind_MACD.MACD_DIF(EMA12 = EMA12, EMA26 = EMA26)
        MACD_DEA    = Fin_Ind_MACD.MACD_DEA(Seq = Seq, MACD_DIF = MACD_DIF)
        MACD_STICK  = Fin_Ind_MACD.MACD_STICK(MACD_DIF = MACD_DIF, MACD_DEA = MACD_DEA)
        KDJ_K       = Fin_Ind_KDJ.KDJ_K(Seq = Seq, RSV_Prd = 9, K_Prd = 3, High = High, Low = Low, Close = Close)
        KDJ_D       = Fin_Ind_KDJ.KDJ_D(Seq = Seq, RSV_Prd = 9, D_Prd = 3, K_Val = KDJ_K)
        KDJ_J       = Fin_Ind_KDJ.KDJ_J(K_Val = KDJ_K, D_Val = KDJ_D)
        ETG_TRS     = Fin_Etg_Theo.Top_Reversal_Shape(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_BRS     = Fin_Etg_Theo.Bottom_Reversal_Shape(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_T_GROUP = Fin_Etg_Theo.Top_Reversal_Shape_s_Group_Top(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_B_GROUP = Fin_Etg_Theo.Bottom_Reversal_Shape_s_Group_Bottom(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        # ..........................................
        JSON_Records_Stocks_Daily_Indicator.append({
            "id":          ID,
            "sma5":        Fin_Ind_SMA_for_005.SMA(Seq = Seq, Period = 5, Close = Close),
            "sma10":       Fin_Ind_SMA_for_010.SMA(Seq = Seq, Period = 10, Close = Close),
            "ema12":       EMA12,
            "ema26":       EMA26,
            "macd_dif":    MACD_DIF,
            "macd_dea":    MACD_DEA,
            "macd_stick":  MACD_STICK,
            "kdj_k":       KDJ_K,
            "kdj_d":       KDJ_D,
            "kdj_j":       KDJ_J,
            "etg_trs":     ETG_TRS,
            "etg_brs":     ETG_BRS,
            "etg_t_group": ETG_T_GROUP,
            "etg_b_group": ETG_B_GROUP
        })
        # ..........................................
        Count = Count + 1
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator, Completed!\n""")

    #############################################################################
    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Daily_Indicator>", Writing Data ##
    #############################################################################

    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Daily_Indicator}", Writing Data...\n""")
    df = pandas.DataFrame(JSON_Records_Stocks_Daily_Indicator)
    # ..............................................
    PGSQL.TRUNCATE_TABLE(DB_Table = DB_TABLE_Stocks_Data_Daily_Indicator, Primary_Key = "id")
    PGSQL.INSERT_ITERATOR.Fit_Pandas_DataFrame(Pandas_DataFrame = df, DB_Table = DB_TABLE_Stocks_Data_Daily_Indicator)
    while (PGSQL.INSERT_ITERATOR.Pub_Index_Mov <= PGSQL.INSERT_ITERATOR.Pub_Index_Max):
        Count:int = PGSQL.INSERT_ITERATOR.Pub_Index_Mov + 1
        Total:int = PGSQL.INSERT_ITERATOR.Pub_Index_Max + 1
        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
        PGSQL.INSERT_ITERATOR.Next()
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Daily_Indicator}", Writing Completed!\n""")

    ############################################
    ## Calculating Stocks Data Weekly Interim ##
    ############################################

    df = Pandas_DataFrame_for_Stocks_Daily.copy()

    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim...\n""")
    df = Stocks_DataFrame_Preprocessing.Before_Converting_Stocks_Daily_to_Weekly(df)
    # ..............................................
    JSON_Records_Stocks_Weekly_Interim:list  = []
    # ..............................................
    Total = df["id"].count()
    Count = 1
    # ..............................................
    for Idx, Row in df.iterrows():
        ID            = Row["id"]
        Seq           = Row["row_num"]        # Temporary Column for Calculation.
        Time          = Row["time"]
        Week_Num      = Row["week_num"]
        ISO_8601_Week = Row["iso_8601_week"]  # Temporary Column for Calculation.
        Code          = Row["code"]
        Open          = Row["open"]
        High          = Row["high"]
        Low           = Row["low"]
        Close         = Row["close"]
        Change        = Row["change"]
        Volume        = Row["volume"]
        # ..........................................
        if (Count == Total or Count % 100 == 0):
            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
            sys.stdout.flush()
        # ..........................................
        JSON_Records_Stocks_Weekly_Interim.append({
            "id":            ID,
            "row_num":       Seq,
            "time":          Stocks_Data_Conv_Daily_to_Weekly.WK_Time(Seq = Seq, DY_Time = Time),
            "week_num":      Week_Num,
            "iso_8601_week": ISO_8601_Week,
            "code":          Code,
            "open":          Stocks_Data_Conv_Daily_to_Weekly.WK_Open(Seq = Seq, DY_Open = Open),
            "high":          Stocks_Data_Conv_Daily_to_Weekly.WK_High(Seq = Seq, DY_High = High),
            "low":           Stocks_Data_Conv_Daily_to_Weekly.WK_Low(Seq = Seq, DY_Low = Low),
            "close":         Stocks_Data_Conv_Daily_to_Weekly.WK_Close(Seq = Seq, DY_Close = Close),
            "change":        Stocks_Data_Conv_Daily_to_Weekly.WK_Change(Seq = Seq, DY_Change = Change),
            "volume":        Stocks_Data_Conv_Daily_to_Weekly.WK_Volume(Seq = Seq, DY_Volume = Volume)
        })
        # ..........................................
        Count = Count + 1
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim, Completed!\n""")

    ############################################################################
    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly_Interim>", Writing Data ##
    ############################################################################

    if (DB_TABLE_Stocks_Data_Weekly_Interim != "<none>"):
        sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Interim}", Writing Data...\n""")
        df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Interim)
        df = df.drop(["row_num", "iso_8601_week"], axis = 1)  # Delete The Temporary Column for Calculation.
        # ..........................................
        PGSQL.TRUNCATE_TABLE(DB_Table = DB_TABLE_Stocks_Data_Weekly_Interim, Primary_Key = "id")
        PGSQL.INSERT_ITERATOR.Fit_Pandas_DataFrame(Pandas_DataFrame = df, DB_Table = DB_TABLE_Stocks_Data_Weekly_Interim)
        while (PGSQL.INSERT_ITERATOR.Pub_Index_Mov <= PGSQL.INSERT_ITERATOR.Pub_Index_Max):
            Count:int = PGSQL.INSERT_ITERATOR.Pub_Index_Mov + 1
            Total:int = PGSQL.INSERT_ITERATOR.Pub_Index_Max + 1
            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
            PGSQL.INSERT_ITERATOR.Next()
        sys.stdout.write(f"""\n""")
        sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Interim}", Writing Completed!\n""")

    ############################################
    ## Aggregating Stocks Data Weekly Interim ##
    ############################################

    sys.stdout.write(f"""[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim...\n""")
    df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Interim)
    df["cum_max"] = df[["iso_8601_week", "row_num"]].groupby("iso_8601_week", as_index = False).cummax()["row_num"]
    df = df.query(""" row_num == cum_max """)
    df["memo"   ] = f"股票/周数据/{Adjusted}"
    df = df.drop(["row_num", "iso_8601_week", "cum_max"], axis = 1)  # Delete The Temporary Column for Calculation.
    sys.stdout.write(f"""[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim, Completed!\n""")

    ####################################################################
    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly>", Writing Data ##
    ####################################################################

    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly}", Writing Data...\n""")
    PGSQL.TRUNCATE_TABLE(DB_Table = DB_TABLE_Stocks_Data_Weekly, Primary_Key = "id")
    PGSQL.INSERT_ITERATOR.Fit_Pandas_DataFrame(Pandas_DataFrame = df, DB_Table = DB_TABLE_Stocks_Data_Weekly)
    while (PGSQL.INSERT_ITERATOR.Pub_Index_Mov <= PGSQL.INSERT_ITERATOR.Pub_Index_Max):
        Count:int = PGSQL.INSERT_ITERATOR.Pub_Index_Mov + 1
        Total:int = PGSQL.INSERT_ITERATOR.Pub_Index_Max + 1
        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
        PGSQL.INSERT_ITERATOR.Next()
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly}", Writing Completed!\n""")

    ########################################
    ## Calculating Stocks Weekly Indicator ##
    ########################################

    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Indicator...\n""")
    df = Stocks_DataFrame_Preprocessing.Before_Calculating_Stocks_Weekly_Indicators(df)
    # ..............................................
    JSON_Records_Stocks_Weekly_Indicator:list = []
    # ..............................................
    Total = df["id"].count()
    Count = 1
    # ..............................................
    for Idx, Row in df.iterrows():
        ID    = Row["id"]
        Seq   = Row["row_num"]
        High  = Row["high"]
        Low   = Row["low"]
        Close = Row["close"]
        # ..........................................
        if (Count == Total or Count % 100 == 0):
            sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
            sys.stdout.flush()
        # ..........................................
        EMA12       = Fin_Ind_EMA_for_012.EMA(Seq = Seq, Period = 12, Close = Close)
        EMA26       = Fin_Ind_EMA_for_026.EMA(Seq = Seq, Period = 26, Close = Close)
        MACD_DIF    = Fin_Ind_MACD.MACD_DIF(EMA12 = EMA12, EMA26 = EMA26)
        MACD_DEA    = Fin_Ind_MACD.MACD_DEA(Seq = Seq, MACD_DIF = MACD_DIF)
        MACD_STICK  = Fin_Ind_MACD.MACD_STICK(MACD_DIF = MACD_DIF, MACD_DEA = MACD_DEA)
        KDJ_K       = Fin_Ind_KDJ.KDJ_K(Seq = Seq, RSV_Prd = 9, K_Prd = 3, High = High, Low = Low, Close = Close)
        KDJ_D       = Fin_Ind_KDJ.KDJ_D(Seq = Seq, RSV_Prd = 9, D_Prd = 3, K_Val = KDJ_K)
        KDJ_J       = Fin_Ind_KDJ.KDJ_J(K_Val = KDJ_K, D_Val = KDJ_D)
        ETG_TRS     = Fin_Etg_Theo.Top_Reversal_Shape(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_BRS     = Fin_Etg_Theo.Bottom_Reversal_Shape(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_T_GROUP = Fin_Etg_Theo.Top_Reversal_Shape_s_Group_Top(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        ETG_B_GROUP = Fin_Etg_Theo.Bottom_Reversal_Shape_s_Group_Bottom(Seq = Seq, UpperEdge = High, LowerEdge = Low)
        # ..........................................
        JSON_Records_Stocks_Weekly_Indicator.append({
            "id":          ID,
            "sma5":        Fin_Ind_SMA_for_005.SMA(Seq = Seq, Period = 5, Close = Close),
            "sma10":       Fin_Ind_SMA_for_010.SMA(Seq = Seq, Period = 10, Close = Close),
            "ema12":       EMA12,
            "ema26":       EMA26,
            "macd_dif":    MACD_DIF,
            "macd_dea":    MACD_DEA,
            "macd_stick":  MACD_STICK,
            "kdj_k":       KDJ_K,
            "kdj_d":       KDJ_D,
            "kdj_j":       KDJ_J,
            "etg_trs":     ETG_TRS,
            "etg_brs":     ETG_BRS,
            "etg_t_group": ETG_T_GROUP,
            "etg_b_group": ETG_B_GROUP
        })
        # ..........................................
        Count = Count + 1
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Calculating Stocks Data Weekly Indicator, Completed!\n""")

    #############################################################################
    ## Towards DB_TABLE "<DB_TABLE_Stocks_Data_Weekly_Indicator>", Writing Data ##
    #############################################################################

    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Indicator}", Writing Data...\n""")
    df = pandas.DataFrame(JSON_Records_Stocks_Weekly_Indicator)
    # ..............................................
    PGSQL.TRUNCATE_TABLE(DB_Table = DB_TABLE_Stocks_Data_Weekly_Indicator, Primary_Key = "id")
    PGSQL.INSERT_ITERATOR.Fit_Pandas_DataFrame(Pandas_DataFrame = df, DB_Table = DB_TABLE_Stocks_Data_Weekly_Indicator)
    while (PGSQL.INSERT_ITERATOR.Pub_Index_Mov <= PGSQL.INSERT_ITERATOR.Pub_Index_Max):
        Count:int = PGSQL.INSERT_ITERATOR.Pub_Index_Mov + 1
        Total:int = PGSQL.INSERT_ITERATOR.Pub_Index_Max + 1
        sys.stdout.write(f"""\r[DEBUG] PROCESSING: {Text_Progress_Bar.Double_Line_Arrow(Count = Count, Total = Total)}""")
        PGSQL.INSERT_ITERATOR.Next()
    sys.stdout.write(f"""\n""")
    sys.stdout.write(f"""[DEBUG] PROCESSING: Towards DB_TABLE "{DB_TABLE_Stocks_Data_Weekly_Indicator}", Writing Completed!\n""")

In [7]:
import time

In [8]:
Time_0 = time.time()
Finance_Calculate_Stocks_Based_on_Daily(df_stocks_daily_bfq)
Time_1 = time.time()
# ..................................................
Time_X:dict = {    "basic_second": (Time_1 - Time_0)}
Time_X.update({"remainder_second": (Time_X["basic_second"] % 60)})
Time_X.update({  "integer_minute": (Time_X["basic_second"] - Time_X["remainder_second"]) / 60})
# ..................................................
sys.stdout.write(f"""[DEBUG] Time Consumption: {Time_X["integer_minute"]} Minute {Time_X["remainder_second"]} Second\n""")

[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1319388/1319388
[DEBUG] PROCESSING: Calculating Stocks Data Daily Indicator, Completed!
[DEBUG] PROCESSING: Towards DB_TABLE "tushare_api_cache_en_us_indicator", Writing Data...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1319388/1319388
[DEBUG] PROCESSING: Towards DB_TABLE "tushare_api_cache_en_us_indicator", Writing Completed!
[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1319388/1319388
[DEBUG] PROCESSING: Calculating Stocks Data Weekly Interim, Completed!
[DEBUG] PROCESSING: Towards DB_TABLE "stocks_en_us_daily_to_weekly_interim", Writing Data...
[DEBUG] PROCESSING: [====================>] Pct: 1.00 | 1319388/1319388
[DEBUG] PROCESSING: Towards DB_TABLE "stocks_en_us_daily_to_weekly_interim", Writing Completed!
[DEBUG] PROCESSING: Aggregating Stocks Data Weekly Interim...
[DEBUG] PR

64